##Create Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS ecommerce_analytics;
USE ecommerce_analytics;

##Create Bronze Events Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bronze_event_logs (
  user_id STRING,
  session_id STRING,
  event_name STRING,
  event_timestamp TIMESTAMP,
  device_type STRING,
  product_id STRING,
  revenue DOUBLE
);

##Create Silver Events Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver_event_logs (
  user_id STRING,
  session_id STRING,
  event_name STRING,
  event_timestamp TIMESTAMP,
  device_type STRING,
  product_id STRING,
  revenue DOUBLE
);

##Create Gold Funnel Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold_funnel (
  user_id STRING,
  device_type STRING,
  view_ts TIMESTAMP,
  atc_ts TIMESTAMP,
  checkout_ts TIMESTAMP,
  purchase_ts TIMESTAMP
); 

###Verify Tables Exist

In [0]:
%sql
SHOW TABLES IN ecommerce_analytics;


database,tableName,isTemporary
ecommerce_analytics,bronze_event_logs,false
ecommerce_analytics,gold_funnel,false
ecommerce_analytics,silver_event_logs,false


###Using the project database


In [0]:
%sql
USE ecommerce_analytics;

###Create the bronze_events_raw table from the uploaded data

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bronze_events_raw AS
SELECT *
FROM workspace.default.events;

num_affected_rows,num_inserted_rows


####Quick Check

In [0]:
%sql
-- Row count should also be ~2,000,000
SELECT COUNT(*) AS rows FROM bronze_events_raw;

-- Peek at columns/nullable types
DESCRIBE TABLE bronze_events_raw;

-- Event distribution (should show view, add_to_cart, purchase, click, bounce)
SELECT event_type, COUNT(*) AS cnt
FROM bronze_events_raw
GROUP BY event_type
ORDER BY cnt DESC;

-- Confirm we can detect Checkout page for deriving 'begin_checkout'
SELECT page_category, COUNT(*) AS cnt
FROM bronze_events_raw
GROUP BY page_category
ORDER BY cnt DESC;

rows
2000000


col_name,data_type,comment
event_id,bigint,null
timestamp,timestamp,null
customer_id,bigint,null
session_id,bigint,null
event_type,string,null
product_id,double,null
device_type,string,null
traffic_source,string,null
campaign_id,bigint,null
page_category,string,null


event_type,cnt
view,1043573
click,379008
add_to_cart,284370
bounce,189922
purchase,103127


page_category,cnt
PLP,601284
PDP,599638
Home,399723
Checkout,199871
Cart,199484


##Create a cleaned view of Bronze (normalize columns)
What this does:

Makes all key text lowercase/trimmed so values are consistent
Makes timestamp a proper TIMESTAMP
Uses user_id as the name for the person (from customer_id)
Fills blank device_type with 'unknown' so device segmentation won’t break

In [0]:
%sql
CREATE OR REPLACE VIEW v_bronze_clean AS
SELECT
  CAST(event_id AS STRING)                           AS event_id,
  TIMESTAMP(timestamp)                               AS event_timestamp,
  CAST(customer_id AS STRING)                        AS user_id,
  CAST(session_id AS STRING)                         AS session_id,
  LOWER(TRIM(event_type))                            AS event_type,
  CAST(product_id AS STRING)                         AS product_id,   -- can be NULL for some events
  CASE
    WHEN device_type IS NULL OR TRIM(device_type) = '' THEN 'unknown'
    ELSE LOWER(TRIM(device_type))
  END                                                AS device_type,
  LOWER(TRIM(traffic_source))                        AS traffic_source,
  CAST(campaign_id AS STRING)                        AS campaign_id,
  UPPER(TRIM(page_category))                         AS page_category, -- e.g., PLP, PDP, CHECKOUT
  CAST(session_duration_sec AS DOUBLE)               AS session_duration_sec,
  TRIM(experiment_group)                             AS experiment_group
FROM bronze_events_raw;

###Derive begin_checkout from page views of the Checkout page
This Dataset does not have an explicit begin_checkout event, but it does have page views where page_category = 'CHECKOUT'. I treated those rows as the user starting checkout.

Built a view that:

Keeps real funnel events: view, add_to_cart, purchase
Adds synthetic begin_checkout rows whenever page_category = 'CHECKOUT'

In [0]:
%sql
CREATE OR REPLACE VIEW v_bronze_funnel_events AS
-- Keep the events we need (as-is)
SELECT
  event_id,
  event_timestamp,
  user_id,
  session_id,
  event_type,
  product_id,
  device_type,
  traffic_source,
  campaign_id,
  session_duration_sec,
  experiment_group
FROM v_bronze_clean
WHERE event_type IN ('view','add_to_cart','purchase')

UNION ALL

-- Add a synthetic 'begin_checkout' where the page was the checkout page
SELECT
  CONCAT('bc_', event_id)                            AS event_id,          -- give it a unique id
  event_timestamp,
  user_id,
  session_id,
  'begin_checkout'                                   AS event_type,
  product_id,
  device_type,
  traffic_source,
  campaign_id,
  session_duration_sec,
  experiment_group
FROM v_bronze_clean
WHERE page_category = 'CHECKOUT';

##Create the Silver table to use for funnels
 Wrote a physical table silver_events that contains only the cleaned, funnel‑ready columns.
 Droping any rows missing the essential pieces (timestamp, user_id, session_id, event_type).

In [0]:
%sql
CREATE OR REPLACE TABLE silver_events AS
SELECT
  event_timestamp,
  user_id,
  session_id,
  event_type,         -- view | add_to_cart | begin_checkout | purchase
  product_id,
  device_type,
  traffic_source,
  campaign_id,
  experiment_group
FROM v_bronze_funnel_events
WHERE event_timestamp IS NOT NULL
  AND user_id IS NOT NULL
  AND session_id IS NOT NULL
  AND event_type IS NOT NULL;

num_affected_rows,num_inserted_rows


###Validating Silver checks

In [0]:
%sql
-- Row count (will be LOWER than 2M, because we filtered to funnel events only)
SELECT COUNT(*) AS silver_rows
FROM silver_events;

silver_rows
1630941


In [0]:
%sql
-- Event distribution should now include 'begin_checkout'
SELECT event_type, COUNT(*) AS cnt
FROM silver_events
GROUP BY event_type
ORDER BY cnt DESC;


event_type,cnt
view,1043573
add_to_cart,284370
begin_checkout,199871
purchase,103127


In [0]:
%sql
-- Device distribution (some 'unknown' is OK if the raw data had blanks)
SELECT device_type, COUNT(*) AS cnt
FROM silver_events
GROUP BY device_type
ORDER BY cnt DESC;

device_type,cnt
mobile,959310
desktop,559056
tablet,79607
unknown,32968


In [0]:
%sql
-- Sanity: timestamps range
SELECT MIN(event_timestamp) AS min_ts, MAX(event_timestamp) AS max_ts
FROM silver_events;

min_ts,max_ts
2021-01-01T00:01:37.000Z,2023-12-31T23:57:50.000Z


In [0]:
%sql
-- Ensure essential columns are non-null now
SELECT
  SUM(CASE WHEN event_timestamp IS NULL THEN 1 ELSE 0 END) AS null_ts,
  SUM(CASE WHEN user_id         IS NULL THEN 1 ELSE 0 END) AS null_user,
  SUM(CASE WHEN session_id      IS NULL THEN 1 ELSE 0 END) AS null_session,
  SUM(CASE WHEN event_type      IS NULL THEN 1 ELSE 0 END) AS null_event
FROM silver_events;

null_ts,null_user,null_session,null_event
0,0,0,0


##Create the gold table

In [0]:
%sql
USE ecommerce_analytics;

CREATE OR REPLACE TABLE gold_funnel AS
WITH first_events AS (
  SELECT
    user_id,
    device_type,
    MIN(CASE WHEN event_type = 'view'            THEN event_timestamp END) AS view_ts,
    MIN(CASE WHEN event_type = 'add_to_cart'     THEN event_timestamp END) AS atc_ts,
    MIN(CASE WHEN event_type = 'begin_checkout'  THEN event_timestamp END) AS checkout_ts,
    MIN(CASE WHEN event_type = 'purchase'        THEN event_timestamp END) AS purchase_ts
  FROM silver_events
  GROUP BY user_id, device_type
)
SELECT * FROM first_events;

num_affected_rows,num_inserted_rows


##Quality Checks: Does Gold Look good?

In [0]:
%sql
SELECT COUNT(*) FROM gold_funnel;

count(1)
276922


In [0]:
%sql
SELECT * FROM gold_funnel LIMIT 10;

user_id,device_type,view_ts,atc_ts,checkout_ts,purchase_ts
91993,mobile,2021-01-01T11:55:30.000Z,2021-06-12T17:42:04.000Z,2022-06-26T22:56:14.000Z,null
48406,mobile,2021-09-18T18:13:27.000Z,2022-11-02T15:47:34.000Z,2022-06-12T06:36:23.000Z,null
15508,mobile,2021-01-21T05:32:04.000Z,null,2022-02-17T22:27:42.000Z,2022-01-09T03:00:03.000Z
19682,mobile,2021-07-28T12:51:44.000Z,2022-12-02T01:57:12.000Z,2023-05-31T02:49:56.000Z,2023-05-31T02:49:56.000Z
25984,desktop,2022-05-01T21:36:26.000Z,null,null,2023-07-28T09:37:53.000Z
2975,tablet,null,2023-09-21T07:38:14.000Z,null,2022-08-10T21:52:35.000Z
93406,mobile,2021-04-01T18:20:37.000Z,2021-05-01T12:32:49.000Z,2022-01-19T21:10:29.000Z,null
24386,desktop,2022-01-09T18:56:46.000Z,2021-10-02T18:36:09.000Z,2023-07-11T07:35:22.000Z,null
3587,tablet,2022-05-07T06:49:06.000Z,null,null,null
89550,mobile,2022-01-17T05:44:31.000Z,2021-02-14T15:28:40.000Z,2022-10-12T10:30:13.000Z,2021-12-24T03:00:31.000Z


##Compute Funnel Conversion Rates (by Device)

Calculate:

- View → Add to Cart
- Add to Cart → Checkout
- Checkout → Purchase
- Overall Conversion

In [0]:
%sql
SELECT
  device_type,
  COUNT(*)                                    AS total_users,
  COUNT(view_ts)                               AS users_viewed,
  COUNT(atc_ts)                                 AS users_atc,
  COUNT(checkout_ts)                            AS users_checkout,
  COUNT(purchase_ts)                            AS users_purchased,

  ROUND(COUNT(atc_ts)      * 100.0 / COUNT(view_ts), 2)        AS view_to_atc,
  ROUND(COUNT(checkout_ts) * 100.0 / COUNT(atc_ts), 2)         AS atc_to_checkout,
  ROUND(COUNT(purchase_ts) * 100.0 / COUNT(checkout_ts), 2)    AS checkout_to_purchase,
  ROUND(COUNT(purchase_ts) * 100.0 / COUNT(view_ts), 2)        AS overall_conversion

FROM gold_funnel
GROUP BY device_type
ORDER BY device_type;

device_type,total_users,users_viewed,users_atc,users_checkout,users_purchased,view_to_atc,atc_to_checkout,checkout_to_purchase,overall_conversion
desktop,99365,97187,62146,49730,29708,63.94,80.02,59.74,30.57
mobile,99988,99772,81358,69273,45252,81.54,85.15,65.32,45.36
tablet,51630,39952,12818,9317,4953,32.08,72.69,53.16,12.40
unknown,25939,18930,5678,4021,2054,29.99,70.82,51.08,10.85


##Build a simple Databricks SQL dashboard
Going to  make 3 visuals you can open anytime:

1.Funnel by device (bars)

2.Drop‑off heatmap (where the leaks are)

3.Weekly conversion trend (how conversion moves over time)

###Create a view for device funnel metrics

In [0]:
%sql
USE ecommerce_analytics;

CREATE OR REPLACE VIEW v_funnel_by_device AS
SELECT
  device_type,
  COUNT(*)                                  AS users_view,
  COUNT(atc_ts)                             AS users_atc,
  COUNT(checkout_ts)                        AS users_checkout,
  COUNT(purchase_ts)                        AS users_purchase,
  ROUND(COUNT(atc_ts)      * 100.0 / NULLIF(COUNT(*),0), 2)  AS view_to_atc,
  ROUND(COUNT(checkout_ts) * 100.0 / NULLIF(COUNT(atc_ts),0), 2) AS atc_to_checkout,
  ROUND(COUNT(purchase_ts) * 100.0 / NULLIF(COUNT(checkout_ts),0), 2) AS checkout_to_purchase,
  ROUND(COUNT(purchase_ts) * 100.0 / NULLIF(COUNT(*),0), 2) AS overall_conversion
FROM gold_funnel
GROUP BY device_type;

###Create a drop‑off view by device


In [0]:
%sql
CREATE OR REPLACE VIEW v_dropoffs_by_device AS
SELECT
  device_type,
  users_view,
  users_atc,
  users_checkout,
  users_purchase,
  (users_view - users_atc)         AS drop_view_to_atc,
  (users_atc - users_checkout)     AS drop_atc_to_checkout,
  (users_checkout - users_purchase) AS drop_checkout_to_purchase
FROM v_funnel_by_device;


In [0]:
%sql
SELECT * FROM v_dropoffs_by_device ORDER BY device_type;


device_type,users_view,users_atc,users_checkout,users_purchase,drop_view_to_atc,drop_atc_to_checkout,drop_checkout_to_purchase
desktop,99365,62146,49730,29708,37219,12416,20022
mobile,99988,81358,69273,45252,18630,12085,24021
tablet,51630,12818,9317,4953,38812,3501,4364
unknown,25939,5678,4021,2054,20261,1657,1967


###Create v_weekly_conv (weekly purchase rate by device)

In [0]:
%sql
CREATE OR REPLACE VIEW v_weekly_conv AS
WITH base AS (
  SELECT
    DATE_TRUNC('week', event_timestamp) AS week_start,
    device_type,
    SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END)     AS views,
    SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS purchases
  FROM silver_events
  GROUP BY 1, 2
)
SELECT
  week_start,
  device_type,
  views,
  purchases,
  ROUND(purchases * 100.0 / NULLIF(views,0), 2) AS purchase_rate_pct
FROM base;

In [0]:
%sql
SELECT * FROM v_weekly_conv ORDER BY week_start, device_type LIMIT 50;

week_start,device_type,views,purchases,purchase_rate_pct
2020-12-28T00:00:00.000Z,desktop,979,98,10.01
2020-12-28T00:00:00.000Z,mobile,1599,158,9.88
2020-12-28T00:00:00.000Z,tablet,129,14,10.85
2020-12-28T00:00:00.000Z,unknown,49,6,12.24
2021-01-04T00:00:00.000Z,desktop,2321,224,9.65
2021-01-04T00:00:00.000Z,mobile,3923,371,9.46
2021-01-04T00:00:00.000Z,tablet,328,39,11.89
2021-01-04T00:00:00.000Z,unknown,132,10,7.58
2021-01-11T00:00:00.000Z,desktop,2324,233,10.03
2021-01-11T00:00:00.000Z,mobile,3927,375,9.55


In [0]:
%sql
CREATE OR REPLACE VIEW v_funnel_by_device AS
SELECT
  device_type,
  COUNT(*) AS users_view,
  COUNT(atc_ts) AS users_atc,
  COUNT(checkout_ts) AS users_checkout,
  COUNT(purchase_ts) AS users_purchase
FROM gold_funnel
GROUP BY device_type;

In [0]:
%sql
SELECT * FROM v_funnel_by_device;

device_type,users_view,users_atc,users_checkout,users_purchase
mobile,99988,81358,69273,45252
desktop,99365,62146,49730,29708
tablet,51630,12818,9317,4953
unknown,25939,5678,4021,2054


In [0]:
%sql
-- Always select your project database first
USE ecommerce_analytics;

-- See the exact table names, including any weird spaces/characters
SHOW TABLES IN ecommerce_analytics;

-- If you want a focused look:
SHOW TABLES LIKE 'gold%';

-- What is the current database Databricks is using in THIS notebook?
SELECT current_database();

database,tableName,isTemporary
ecommerce_analytics,bronze_event_logs,false
ecommerce_analytics,bronze_events_raw,false
ecommerce_analytics,gold_funnel,false
ecommerce_analytics,silver_event_logs,false
ecommerce_analytics,silver_events,false
ecommerce_analytics,v_bronze_clean,false
ecommerce_analytics,v_bronze_funnel_events,false
ecommerce_analytics,v_dropoffs_by_device,false
ecommerce_analytics,v_funnel_by_device,false
ecommerce_analytics,v_weekly_conv,false


database,tableName,isTemporary


current_schema()
ecommerce_analytics


In [0]:
%sql
DESCRIBE TABLE gold_funnel;

-- Peek at a few rows
SELECT * FROM gold_funnel LIMIT 5;


col_name,data_type,comment
user_id,string,null
device_type,string,null
view_ts,timestamp,null
atc_ts,timestamp,null
checkout_ts,timestamp,null
purchase_ts,timestamp,null


user_id,device_type,view_ts,atc_ts,checkout_ts,purchase_ts
91993,mobile,2021-01-01T11:55:30.000Z,2021-06-12T17:42:04.000Z,2022-06-26T22:56:14.000Z,null
48406,mobile,2021-09-18T18:13:27.000Z,2022-11-02T15:47:34.000Z,2022-06-12T06:36:23.000Z,null
15508,mobile,2021-01-21T05:32:04.000Z,null,2022-02-17T22:27:42.000Z,2022-01-09T03:00:03.000Z
19682,mobile,2021-07-28T12:51:44.000Z,2022-12-02T01:57:12.000Z,2023-05-31T02:49:56.000Z,2023-05-31T02:49:56.000Z
25984,desktop,2022-05-01T21:36:26.000Z,null,null,2023-07-28T09:37:53.000Z


In [0]:
%sql
USE ecommerce_analytics;

-- 1) Build a gold-style table per user + source + device
CREATE OR REPLACE TABLE gold_funnel_source AS
WITH first_events AS (
  SELECT
    user_id,
    LOWER(COALESCE(traffic_source,'unknown')) AS traffic_source,
    LOWER(COALESCE(device_type,'unknown'))    AS device_type,
    MIN(CASE WHEN event_type = 'view'           THEN event_timestamp END) AS view_ts,
    MIN(CASE WHEN event_type = 'add_to_cart'    THEN event_timestamp END) AS atc_ts,
    MIN(CASE WHEN event_type = 'begin_checkout' THEN event_timestamp END) AS checkout_ts,
    MIN(CASE WHEN event_type = 'purchase'       THEN event_timestamp END) AS purchase_ts
  FROM silver_event_logs         -- your Silver table
  GROUP BY user_id, LOWER(COALESCE(traffic_source,'unknown')), LOWER(COALESCE(device_type,'unknown'))
)
SELECT * FROM first_events;

-- 2) Conversion by traffic source (and device, if you want)
CREATE OR REPLACE VIEW v_funnel_by_source AS
SELECT
  traffic_source,
  device_type,
  COUNT(*)                                    AS users_view,
  COUNT(atc_ts)                                AS users_atc,
  COUNT(checkout_ts)                           AS users_checkout,
  COUNT(purchase_ts)                           AS users_purchased,
  ROUND(COUNT(atc_ts)      * 100.0 / NULLIF(COUNT(*),0), 2)   AS view_to_atc,
  ROUND(COUNT(checkout_ts) * 100.0 / NULLIF(COUNT(atc_ts),0), 2) AS atc_to_checkout,
  ROUND(COUNT(purchase_ts) * 100.0 / NULLIF(COUNT(checkout_ts),0), 2) AS checkout_to_purchase,
  ROUND(COUNT(purchase_ts) * 100.0 / NULLIF(COUNT(*),0), 2)   AS overall_conversion
FROM gold_funnel_source
GROUP BY traffic_source, device_type
ORDER BY traffic_source, device_type;

-- 3) Quick check
SELECT * FROM v_funnel_by_source LIMIT 50;